# B0829024 HW

## Modern convnet architecture patterns

### Modularity, hierarchy, and reuse

### Residual connections

**Residual block where the number of filters changes**

In [51]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu")(inputs)
residual = x
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
residual = layers.Conv2D(64, 1)(residual)
x = layers.add([x, residual])

**Case where target block includes a max pooling layer**

In [52]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu")(inputs)
residual = x
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(2, padding="same")(x)
residual = layers.Conv2D(64, 1, strides=2)(residual)
x = layers.add([x, residual])

In [53]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Rescaling(1./255)(inputs)

def residual_block(x, filters, pooling=False):
    residual = x
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    if pooling:
        x = layers.MaxPooling2D(2, padding="same")(x)
        residual = layers.Conv2D(filters, 1, strides=2)(residual)
    elif filters != residual.shape[-1]:
        residual = layers.Conv2D(filters, 1)(residual)
    x = layers.add([x, residual])
    return x

x = residual_block(x, filters=32, pooling=True)
x = residual_block(x, filters=64, pooling=True)
x = residual_block(x, filters=128, pooling=False)

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_30 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling_9 (Rescaling)        (None, 32, 32, 3)    0           ['input_30[0][0]']               
                                                                                                  
 conv2d_141 (Conv2D)            (None, 32, 32, 32)   896         ['rescaling_9[0][0]']            
                                                                                                  
 conv2d_142 (Conv2D)            (None, 32, 32, 32)   9248        ['conv2d_141[0][0]']             
                                                                                            

### Batch normalization

### Depthwise separable convolutions

### Putting it together: A mini Xception-like model

In [54]:
# from google.colab import files
# files.upload()

In [55]:
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c dogs-vs-cats
# !unzip -qq train.zip

In [56]:
import os, shutil, pathlib
from tensorflow.keras.utils import image_dataset_from_directory

# original_dir = pathlib.Path("train")
# new_base_dir = pathlib.Path("cats_vs_dogs_small")

original_dir = pathlib.Path('E:\Code\Github\PatternRecognition_HW\HW08\train')
new_base_dir = pathlib.Path('E:\Code\Github\PatternRecognition_HW\HW08\cats_vs_dogs_small')

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

# make_subset("train", start_index=0, end_index=1000)
# make_subset("validation", start_index=1000, end_index=1500)
# make_subset("test", start_index=1500, end_index=2500)

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [57]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [58]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)

x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32, 64, 128, 256, 512]:
    residual = x

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(
        size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [59]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset)

Epoch 1/100
63/63 [==============================] - 41s 537ms/step - loss: 0.7135 - accuracy: 0.5725 - val_loss: 0.7073 - val_accuracy: 0.5000
Epoch 2/100
63/63 [==============================] - 27s 422ms/step - loss: 0.6593 - accuracy: 0.5925 - val_loss: 0.6987 - val_accuracy: 0.5000
Epoch 3/100
63/63 [==============================] - 32s 499ms/step - loss: 0.6521 - accuracy: 0.6200 - val_loss: 0.7032 - val_accuracy: 0.5000
Epoch 4/100
63/63 [==============================] - 32s 507ms/step - loss: 0.6189 - accuracy: 0.6655 - val_loss: 0.6993 - val_accuracy: 0.5000
Epoch 5/100
63/63 [==============================] - 31s 496ms/step - loss: 0.5919 - accuracy: 0.6930 - val_loss: 0.7272 - val_accuracy: 0.5000
Epoch 6/100
63/63 [==============================] - 32s 500ms/step - loss: 0.5745 - accuracy: 0.7025 - val_loss: 0.7131 - val_accuracy: 0.4990
Epoch 7/100
63/63 [==============================] - 32s 506ms/step - loss: 0.5500 - accuracy: 0.7305 - val_loss: 0.9001 - val_accuracy: